# Example Notebook for Time-Order Map for Seamless Zooming

This notebook can be used to test out the python modules in this project.

**Initialization**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# add packages from 'src'
import sys
from pathlib import Path

# App directory
project_root = Path.cwd().parent  # Adjust this if necessary
sys.path.append(str(project_root))

In [ ]:
# Package Modules
from src.utils.data_exporting import *
from src.utils.data_importing import *
from src.utils.data_processing import *
from src.algo.global_ranking import *
from src.orchestrator import *

In [ ]:
# further packages
import matplotlib.pyplot as plt
import pandas as pd
import pm4py
import seaborn as sns

**Data import**

In [ ]:
# Evaluation
df_org = load_event_log("runningexample.xes", "evaluation_data")
df = df_org.copy()

**Data pre-processing**

*Filter by Cases*

In [ ]:
#cases = df["case:concept:name"].unique()
#casessorted = sorted(cases)
#casessorted_filtered = casessorted[0:1050]
#len(casessorted_filtered)
#df_filtered = df[df["case:concept:name"].isin(casessorted_filtered)]

*Prepare log for D3.js*

In [ ]:
df = process_log_for_d3js(df)

*-- Break down process_log_for_d3js*

In [ ]:
df = simplifyLog(df)
df = relativeTimestamps(df)

In [ ]:
#global_ranking_method_df_relativetime(df, annotation=True)

In [ ]:
#df, ranking_map = global_ranking_of_eventdata(df)

**Export log**

In [ ]:
file_path = "data/export_data"
file_name = "data-runningexample.csv"

*CSV*

In [ ]:
#df.to_csv(file_path + file_name)

*XES*

In [ ]:
file_path = "export_data/"
file_name = "roadtrafficfine_1050.xes"
#export_event_log(df, filename= file_name, foldername=file_path)

### Additional Visaulizations in Paper

**Dotted Chart**

In [ ]:
import plotly.express as px

# Sort log
log = df.sort_values(by=['time:timestamp', 'case:concept:name'])
log['time:relative:days'] = pd.to_timedelta(log['time:timestamp:relative'], unit='ns') / pd.Timedelta(days=1)

# Create a scatter plot for a dotted chart
fig = px.scatter(
    log,
    x='time:relative:days',
    y='case:concept:name',
    color='concept:name',
    opacity=0.9,
    width=700,
    height=320,
)

# Customize layout
fig.update_layout(
    xaxis_title="Relative Time (days)",
    #xaxis_title="Time",
    yaxis_title="Cases",
    legend_title="Activities",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    plot_bgcolor='#f2f2f2',
)

# Visualizate the plot
#fig.update_traces(marker=dict(size=10))
fig.update_traces(marker=dict(size=10))
fig.show()

# Write image
file_path = f"{project_root}/data/"
file_name="fig_ex-instance-running"
file_suffix=".svg"
fig.write_image(file_path+file_name+file_suffix)

**Performance DFG**

In [ ]:
from pm4py.visualization.dfg import visualizer

log = df.copy()

performance_dfg, start_activities, end_activities = pm4py.discover_performance_dfg(log)

# Create the visualization object (gviz)
gviz = visualizer.apply(performance_dfg,
                        variant=visualizer.Variants.PERFORMANCE,
                        parameters={"start_activities": start_activities,
                                    "end_activities": end_activities})
gviz

In [ ]:
import graphviz

#filepath
file_path = f"{project_root}/data/"
file_name="fig_ex-model-running"
file_suffix=".svg"

#export DiGraph
gviz.render(file_path+file_name, format='svg', cleanup=True)